
<div style="background-image: linear-gradient(to right, #4b4cff , #00d4ff); text-align: center; padding: 50px;">
  <h1 style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; font-size: 48px; color: white; text-shadow: 2px 2px #4b4cff;">
  Spaceship Titanic
  </h1>
</div>

<h2 style="color:#f7ce5b; font-family: Arial, sans-serif; font-size: 40px; margin-top: 50px; border-bottom: 2px solid #f7ce5b;">
Library
</h2>

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib
import seaborn as sns
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

<h2 style="color:#f7ce5b; font-family: Arial, sans-serif; font-size: 40px; margin-top: 50px; border-bottom: 2px solid #f7ce5b;">
Data
</h2>

In [ ]:
data_train = pd.read_csv("train.csv")

data_test = pd.read_csv("test.csv")
data_train

df = data_train.copy(deep=True)

<h2 style="color:#f7ce5b; font-family: Arial, sans-serif; font-size: 40px; margin-top: 50px; border-bottom: 2px solid #f7ce5b;">
First Regard
</h2>

In [ ]:
data_train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

<h2 style="color:#f7ce5b; font-family: Arial, sans-serif; font-size: 40px; margin-top: 50px; border-bottom: 2px solid #f7ce5b;">
Data Treatment
</h2>

In [ ]:
# If passenger is in cryosleep then all the amenities are 0
col_list = ['RoomService', 'FoodCourt', 'ShoppingMall', 'VRDeck','Spa']
df.loc[df.CryoSleep.eq(True), col_list] = 0

In [ ]:
# Where summ of amenities is 0 we also impute 0 for missing values
zero_amenities = df[col_list].sum(axis=1).eq(0)
df.loc[zero_amenities, col_list] = 0

In [ ]:
# If the passenger is not in cryosleep then we put the median ofthe expenses of each amenitie for people not in cryosleep
for i in col_list: 
    df.loc[df[i].isna(), i] = df.loc[df[i].gt(0), i].median()

In [ ]:
#Summ of all the expenses
df['Gold'] = df[col_list].sum(axis = 1)

In [ ]:
## If Gold>0 and we don not know if he is in cryolseep then we set cryolseep to false
df.loc[(df.CryoSleep.isna() & df.Gold.gt(0)), 'CryoSleep']=False

In [ ]:
#in the data we have 3653 people with gold==0 and out of these 3653, about 3000 of the are in cryosleep which is about 83% so 
#we simulate a random distribuition where if it s greater than 0.83 it is not in cryosleep otherwise it is 
def fonct_cryo(x):
    if x>=0.83:
        return False
    return True

df.loc[(df.CryoSleep.isna()& df.Gold==0),'CryoSleep']=fonct_cryo(np.random.random())

In [ ]:
df.groupby(['HomePlanet', 'Destination']).VIP.value_counts()

HomePlanet  Destination    VIP  
Earth       55 Cancri e    False     668
            PSO J318.5-22  False     693
            TRAPPIST-1e    False    3030
Europa      55 Cancri e    False     807
                           True       63
            PSO J318.5-22  True       10
                           False       9
            TRAPPIST-1e    False    1107
                           True       56
Mars        55 Cancri e    False     189
            PSO J318.5-22  False      39
                           True        8
            TRAPPIST-1e    False    1383
                           True       55
Name: VIP, dtype: int64

In [ ]:
df.groupby('VIP').Age.min()

On remarque ici que Il n'y a aucun VIP qui viennent de Earth
Il n'y a pas de vip de Mars vers Cancri
Que le plus jeune VIP a 18 ans 


In [ ]:
# We know that the youngest VIP is 18 years old
df.loc[(df.VIP.isna() & (df.Age < 18)), 'VIP'] = False

In [ ]:
# We know that there are no VIP's comming from earth
df.loc[(df.VIP.isna() & (df.HomePlanet == 'Earth')), 'VIP'] = False

# We know that there are no VIP's comming Mars and going to Cancri
df.loc[(df.VIP.isna() 
        & (df.HomePlanet.eq('Mars'))
        & (df.Destination.eq('55 Cancri e'))), 'VIP'] = False

In [ ]:
df[['Cabin_deck', 'Cabin_num', 'Cabin_side']] = df.Cabin.str.split("/", expand=True)

In [ ]:
df.groupby(['VIP', 'CryoSleep']).Deck.value_counts()

On remarque ici qu'il n'y a pas de VIP sur les deck G et T

Les VIP qui sont en Cryosleep sont seulement sur les decks A, B, C, D

In [ ]:
# We know that there are no VIP passengers on deck G and T
df.loc[(df.VIP.isna() 
        & (df.Cabin_deck.isin(['G', 'T']))), 'VIP'] = False

# We Know that people who are not on decks A to D and not in CryoSleep are VIP's
df.loc[df.VIP.isna() 
       & df.CryoSleep.eq(False) 
       & ~df.Cabin_deck.isin(['A', 'B', 'C', 'D']), 'VIP'] = True


#For the rest just put nan VIPs to false as probability is low to  be a vip
df.loc[df.VIP.isna() , 'VIP'] = False

In [ ]:
# The VIPs going to Cancri are just from Europa
df.loc[(df.HomePlanet.isna() 
        & df.VIP.eq(True) 
        & df.Destination.eq('55 Cancri e')), 'HomePlanet'] = 'Europa'

In [ ]:
# Impute HomePlanet based on previous distribution analysis
df.loc[(df.HomePlanet.isna() 
        & df.Cabin_deck.isin(['T', 'A', 'B', 'C',])), 'HomePlanet'] = 'Europa'

df.loc[(df.HomePlanet.isna() & df.Cabin_deck.eq('G')), 'HomePlanet'] = 'Earth'

df.loc[(df.HomePlanet.isna() & df.Cabin_deck.eq('G')), 'HomePlanet'] = 'Earth'

In [ ]:
df.loc[df.HomePlanet.isna(), 'HomePlanet']

59      NaN
186     NaN
225     NaN
291     NaN
405     NaN
       ... 
8489    NaN
8515    NaN
8613    NaN
8666    NaN
8674    NaN
Name: HomePlanet, Length: 107, dtype: object

In [ ]:
df.groupby('Destination').count()

,PassengerId,HomePlanet,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Gold,Cabin_deck,Cabin_num,Cabin_side
Destination,,,,,,,,,,,,,,,,,
55 Cancri e,1800,1788,1800,1750,1766,1800,1800,1800,1800,1800,1800,1757,1800,1800,1750,1750,1750
PSO J318.5-22,796,790,796,778,782,796,796,796,796,796,796,779,796,796,778,778,778
TRAPPIST-1e,5915,5827,5915,5788,5787,5915,5915,5915,5915,5915,5915,5779,5915,5915,5788,5788,5788


In [ ]:
# Impute median Age for people with VIP status
df.loc[((df.VIP == True) & df.Age.isna()), 'Age'] = df.loc[(df.VIP == True), 'Age'].median()

# Impute median Age for people that have expenses
df.loc[(df.Age.isna() & df.Gold.gt(0)), 'Age'] = df.loc[df.Gold.gt(0), 'Age'].median()

# Impute median Age for people with no expenses and not in CryoSleep
df.loc[(df.Age.isna() & df.Gold.eq(0) & df.CryoSleep.eq(False)), 'Age'] = df.loc[(df.Gold.eq(0) & df.CryoSleep.eq(False)), 'Age'].median()

# Impute an overall median Age for people not included in previous groups
df.Age.fillna(df.Age.median(), inplace=True)

In [ ]:
df.groupby('HomePlanet').count()

,PassengerId,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Gold,Cabin_deck,Cabin_num,Cabin_side
HomePlanet,,,,,,,,,,,,,,,,,
Earth,4663,4663,4568,4561,4663,4663,4663,4663,4663,4663,4663,4555,4663,4663,4568,4568,4568
Europa,2164,2164,2103,2127,2164,2164,2164,2164,2164,2164,2164,2118,2164,2164,2103,2103,2103
Mars,1759,1759,1722,1717,1759,1759,1759,1759,1759,1759,1759,1718,1759,1759,1722,1722,1722


In [ ]:
# We drop these variables to avoid overfitting of cabin num as it is not relevant
df = df.drop(['PassengerId', 'Name','Cabin_num'], axis = 1)

In [ ]:
# For the remaining missing values Of Home Planet we input the most likely planet of destination and home planet
df.loc[df.HomePlanet.isna(),'HomePlanet']='Earth'
df.loc[df.Destination.isna(),'Destination']='TRAPPIST-1e'

In [ ]:
#Encodage One hot 

df['HomePlanet_Europa'] = df.apply(lambda x : 1 if x['HomePlanet'] == 'Europa' else 0 , axis= 1)
df['HomePlanet_Earth'] = df.apply(lambda x : 1 if x['HomePlanet'] == 'Earth' else 0 , axis= 1)
df['HomePlanet_Mars'] = df.apply(lambda x : 1 if x['HomePlanet'] == 'Mars' else 0 , axis= 1)
df['Destination_TRAPPIST'] = df.apply(lambda x : 1 if x['Destination'] == 'TRAPPIST-1e' else 0 , axis= 1)
df['Destination_55_Cancri'] = df.apply(lambda x : 1 if x['Destination'] == '55 Cancri e' else 0 , axis= 1)
df['Destination_MPSO'] = df.apply(lambda x : 1 if x['Destination'] == 'PSO J318.5-22' else 0 , axis= 1)

decks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']

#Ainsi ici on n'a pas trouvé de moyen de retrouver les cabins manquantes, on les mets donc toutes à 0 dans l'encodage One hot
for deck in decks : 
    df[f'Cabin_deck{deck}'] = df.apply(lambda x: 1 if x['Cabin_deck'] == f'{deck}' else 0, axis = 1)


In [ ]:
#Encodage one hot
df['CryoSleep'] = LabelEncoder().fit_transform(df['CryoSleep'])
df['VIP'] = LabelEncoder().fit_transform(df['VIP'])

# 0 for side P and 1 for side S, if it is a missning value we put it to 0 for side P
df['Cabin_side'] = LabelEncoder().fit_transform(df['Cabin_side'])

In [ ]:
df = df.drop([ 'Destination', 'HomePlanet', 'Cabin', 'Cabin_deck', 'Transported'], axis = 1)

In [ ]:
#We resume all our data treatment in this function

def treating_data(data):
    df=df = data.copy(deep=True)

    #Taking care of the missing values in the amenities
    col_list = ['RoomService', 'FoodCourt', 'ShoppingMall', 'VRDeck','Spa']
    df.loc[df.CryoSleep.eq(True), col_list] = 0

    zero_amenities = df[col_list].sum(axis=1).eq(0)
    df.loc[zero_amenities, col_list] = 0

    for i in col_list: 
        df.loc[df[i].isna(), i] = df.loc[df[i].gt(0), i].median()

    df['Gold'] = df[col_list].sum(axis = 1)

    #Taking care of people in Cryolseep

    df.loc[(df.CryoSleep.isna() & df.Gold.gt(0)), 'CryoSleep']=False

    df.loc[(df.CryoSleep.isna()& df.Gold==0),'CryoSleep']=fonct_cryo(np.random.random())

    #Taking care of the VIP's

    df.loc[(df.VIP.isna() & (df.Age < 18)), 'VIP'] = False

    df.loc[(df.VIP.isna() & (df.HomePlanet == 'Earth')), 'VIP'] = False

    df.loc[(df.VIP.isna() 
            & (df.HomePlanet.eq('Mars'))
            & (df.Destination.eq('55 Cancri e'))), 'VIP'] = False
    

    df[['Cabin_deck', 'Cabin_num', 'Cabin_side']] = df.Cabin.str.split("/", expand=True)

    df.loc[(df.VIP.isna() 
        & (df.Cabin_deck.isin(['G', 'T']))), 'VIP'] = False

    df.loc[df.VIP.isna() 
       & df.CryoSleep.eq(False) 
       & ~df.Cabin_deck.isin(['A', 'B', 'C', 'D']), 'VIP'] = True



    df.loc[df.VIP.isna() , 'VIP'] = False


    #Taking care of the Age
    df.loc[((df.VIP == True) & df.Age.isna()), 'Age'] = df.loc[(df.VIP == True), 'Age'].median()

    df.loc[(df.Age.isna() 
        & df.Gold.gt(0)), 'Age'] = df.loc[df.Gold.gt(0), 'Age'].median()

    df.loc[(df.Age.isna() 
        & df.Gold.eq(0)
        & df.CryoSleep.eq(False)), 'Age'] = df.loc[(df.Gold.eq(0) 
                                                    & df.CryoSleep.eq(False)), 'Age'].median()

    df.Age.fillna(df.Age.median(), inplace=True)    

    df = df.drop(['PassengerId', 'Name','Cabin_num'], axis = 1)

    #Taking care of the HomePlanet and Destination

    df.loc[(df.HomePlanet.isna() 
        & df.VIP.eq(True) 
        & df.Destination.eq('55 Cancri e')), 'HomePlanet'] = 'Europa'
    
    df.loc[(df.HomePlanet.isna() 
        & df.Cabin_deck.isin(['T', 'A', 'B', 'C',])), 'HomePlanet'] = 'Europa'

    df.loc[(df.HomePlanet.isna() & df.Cabin_deck.eq('G')), 'HomePlanet'] = 'Earth'

    df.loc[(df.HomePlanet.isna() & df.Cabin_deck.eq('G')), 'HomePlanet'] = 'Earth'

    df.loc[df.HomePlanet.isna(),'HomePlanet']='Earth'
    df.loc[df.Destination.isna(),'Destination']='TRAPPIST-1e'


    #encodage One hot
    df['HomePlanet_Europa'] = df.apply(lambda x : 1 if x['HomePlanet'] == 'Europa' else 0 , axis= 1)
    df['HomePlanet_Earth'] = df.apply(lambda x : 1 if x['HomePlanet'] == 'Earth' else 0 , axis= 1)
    df['HomePlanet_Mars'] = df.apply(lambda x : 1 if x['HomePlanet'] == 'Mars' else 0 , axis= 1)
    df['Destination_TRAPPIST'] = df.apply(lambda x : 1 if x['Destination'] == 'TRAPPIST-1e' else 0 , axis= 1)
    df['Destination_55_Cancri'] = df.apply(lambda x : 1 if x['Destination'] == '55 Cancri e' else 0 , axis= 1)
    df['Destination_MPSO'] = df.apply(lambda x : 1 if x['Destination'] == 'PSO J318.5-22' else 0 , axis= 1)

    decks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
    for deck in decks : 
        df[f'Cabin_deck{deck}'] = df.apply(lambda x: 1 if x['Cabin_deck'] == f'{deck}' else 0, axis = 1)


    df['CryoSleep'] = LabelEncoder().fit_transform(df['CryoSleep'])
    df['VIP'] = LabelEncoder().fit_transform(df['VIP'])
    df['Cabin_side'] = LabelEncoder().fit_transform(df['Cabin_side'])

    df = df.drop([ 'Destination', 'HomePlanet', 'Cabin', 'Cabin_deck'], axis = 1)


    return df

<h2 style="color:#f7ce5b; font-family: Arial, sans-serif; font-size: 40px; margin-top: 50px; border-bottom: 2px solid #f7ce5b;">
Train Test Split
</h2>

In [ ]:
X_train=treating_data(data_train)

In [ ]:
Y_train=X_train['Transported']

X_train=X_train.drop(['Transported'],axis=1)

In [ ]:
X_test=treating_data(data_test)

<h2 style="color:#0039e6; font-family: Arial, sans-serif; font-size: 28px; text-align : center; margin-top: 50px; font-weight: normal;">
Gradient Boosting
</h2> 

In [194]:
from sklearn.ensemble import GradientBoostingClassifier

gb1 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=0)


gb1.fit(X_train, Y_train)


Y_test_gradboost1=gb1.predict(X_test)


df_gradboost1 = pd.DataFrame(data_test['PassengerId'])

df_gradboost1 = df_gradboost1.assign(Transported=Y_test_gradboost1)

df_gradboost1['Transported']=df_gradboost1['Transported'].replace([0,1],[False,True])
# Enregistrer le DataFrame en CSV
df_gradboost1.to_csv('test_spaceship_gradboost_personnal_cleaning.csv', index=False)

#Avec ca on a un score de 0.80196

<h2 style="color:#0039e6; font-family: Arial, sans-serif; font-size: 28px; text-align : center; margin-top: 50px; font-weight: normal;">
Light Boost
</h2>

In [ ]:
import lightgbm as lgb
from sklearn import datasets
from sklearn import metrics

lgb_train = lgb.Dataset(X_train, Y_train)

params = {
    'objective': 'binary',
    'learning_rate' : 0.1,
    'num_leaves': 4,
    'verbose': 1
} 

clf = lgb.LGBMClassifier()
model_lgb = clf.fit(X_train, Y_train)

res_lgb = model_lgb.predict(X_test)

<h2 style="color:#0039e6; font-family: Arial, sans-serif; font-size: 28px; text-align : center; margin-top: 50px; font-weight: normal;">
CatBoost
</h2>

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV

X_train2 = X_train

X_test2 = X_test

Y_train2=Y_train


mod=CatBoostClassifier(depth=6,
                       iterations=600,
                       learning_rate=0.0415,
                       random_strength=9,
                       l2_leaf_reg=7)
mod.fit(X_train2,Y_train2)


resultat=mod.predict(X_test2)

parameters = {'learning_rate': np.random.uniform(0, 1),
              'random_strength': np.random.randint(0, 10),
              'iterations': np.random.randint(100, 1000),
              'depth': np.random.randint(4, 10),
              'l2_leaf_reg': np.random.randint(2, 10)
              }


randm_src = RandomizedSearchCV(estimator=mod, scoring='accuracy', param_distributions = parameters,
                               cv = 2, n_iter = 10, n_jobs=-1)
randm_src.fit(X_train2,Y_train2)

print(" Results from Random Search " )
print("\n The best estimator across ALL searched params:\n", randm_src.bestestimator)
print("\n The best score across ALL searched params:\n", randm_src.bestscore)
print("\n The best parameters across ALL searched params:\n", randm_src.bestparams)

<h2 style="color:#0039e6; font-family: Arial, sans-serif; font-size: 28px; text-align : center; margin-top: 50px; font-weight: normal;">
Random Forest
</h2> 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train1 = X_train

X_test1 = X_test

parametersRF = {'min_samples_split'    : np.random.randint(20,100),
              'min_samples_leaf'       : np.random.randint(50,200),
              'max_depth'              : np.random.randint(4, 10),
              'max_features'           : np.random.randint(2, 7),
              'n_estimators'           : np.random.randint(200, 1000),
              'max_leaf_nodes'         : np.random.randint(50,250)
                 }

modele_rf = RandomForestClassifier(
     n_estimators=140,
     max_depth=5,
     min_samples_split=50,
     min_samples_leaf=250,
     max_features='sqrt',
     max_leaf_nodes=200,
     max_samples=None,
     bootstrap=True)

randm_src = RandomizedSearchCV(estimator=modele_rf, scoring='accuracy', param_distributions = parametersRF,
                               cv = 5, n_iter = 10, n_jobs=-1)
randm_src.fit(X_train1,Y_train)

print(" Results from Random Search " )
print("\n The best estimator across ALL searched params:\n", randm_src.best_estimator_)
print("\n The best score across ALL searched params:\n", randm_src.best_score_)
print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)

modele_rf.fit(X_train1, Y_train)

pred=modele_rf.predict(X_test1)

<div style="background-image: linear-gradient(to right, #4b4cff , #00d4ff); text-align: center; padding: 50px;">
  <h1 style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; font-size: 48px; color: white; text-shadow: 2px 2px #4b4cff;">
  Submission
  </h1>
</div>

In [195]:
X_train.to_csv("trained_data.csv", index= False)
Y_train.to_csv("Y_trained_data.csv", index= False)
X_test.to_csv("test_for_trained_data.csv", index= False)

CryoSleep                0
Age                      0
VIP                      0
RoomService              0
FoodCourt                0
ShoppingMall             0
Spa                      0
VRDeck                   0
Gold                     0
Cabin_side               0
HomePlanet_Europa        0
HomePlanet_Earth         0
HomePlanet_Mars          0
Destination_TRAPPIST     0
Destination_55_Cancri    0
Destination_MPSO         0
Cabin_deckA              0
Cabin_deckB              0
Cabin_deckC              0
Cabin_deckD              0
Cabin_deckE              0
Cabin_deckF              0
Cabin_deckG              0
Cabin_deckT              0
dtype: int64